In [10]:
import numpy as np
import pytorch_finufft
import torch

from typing import Optional

In [11]:
c = torch.randn(5, dtype=torch.complex64, requires_grad=True)
x = torch.arange(5, dtype=torch.float32) * 2 * np.pi / 5

In [13]:
class finufft1D1(torch.autograd.Function):
    @staticmethod
    def forward(
        ctx,
        points,
        values,
        output_shape,
        fftshift: bool = False,
        **finufftkwargs: Optional[str],
    ):
        if output_shape is None and out is None:
            output_shape = len(points)

        finufftkwargs = {k: v for k, v in finufftkwargs.items()}

    @staticmethod
    def backward(ctx, grad_output):
        _i_sign = ctx.isign
        _mode_ordering = ctx.mode_ordering
        finufftkwargs = ctx.finufftkwargs

        points, values = ctx.saved_tensors
        grad_points = grad_values = None

        if ctx.needs_input_grad[0]:
            # w.r.t. the points x_j
            grad_points = None  # finufft.nufft1d2()
        if ctx.needs_input_grad[1]:
            # w.r.t. the values c_j
            np_points = points.detach().numpy()
            np_grad_output = grad_output.numpy()
            print(np_points.dtype)
            print(np_grad_output.dtype)

            grad_values = pytorch_finufft.functional.finufft1D2.apply(
                points,
                grad_output,
                isign=_i_sign,
                modeord=_mode_ordering,
                **finufftkwargs,
            )

        return grad_points, grad_values, None, None, None, None

In [14]:
model = pytorch_finufft.functional.finufft1D1()

/Users/matthewmeeker/miniconda3/envs/PytorchFinufftDev/lib/python3.9/site-packages/torch/autograd/function.py:445: DeprecationWarning: <class '__main__.finufft1D1'> should not be instantiated. Methods on autograd functionsare all static, so you should invoke them on the class itself. Instantiating an autograd function will raise an error in a future version of PyTorch.
  warnings.warn(f"{cls} should not be instantiated. Methods on autograd functions"


In [15]:
finufft_out = model.apply(x, c)

T = 1e-07

w = torch.zeros(5, dtype=torch.float32)
rind = np.random.randint(5)
w[rind] = 1

TypeError: forward() missing 1 required positional argument: 'output_shape'

In [6]:
assert finufft_out.dtype is torch.complex64

In [7]:
gF = torch.norm(torch.abs(finufft_out))

In [8]:
gF.backward()

tensor([ 0.0461-0.2714j,  0.2770-0.0937j,  0.2438+0.1615j,  0.6268-0.2984j,
        -0.5017-0.1401j])
complex64
complex64


TypeError: apply() takes no keyword arguments

In [34]:
gF.dtype

torch.float32

In [9]:
gF

tensor(3.8660, grad_fn=<DotBackward0>)

In [10]:
finufft_out

tensor([ 0.5733-0.9889j, -2.2670-0.3075j, -0.9925+3.7364j, -0.2208-0.5198j,
         0.9294-1.3498j], grad_fn=<finufft1D1Backward>)

In [11]:
torch.abs(finufft_out)

tensor([1.1430, 2.2878, 3.8660, 0.5647, 1.6389], grad_fn=<AbsBackward0>)